In [2]:
import pandas as pd

df_train = pd.read_csv('datos/eliminar/train.csv')
df = pd.read_csv('datos/eliminar/df.csv')

In [ ]:
df_train

In [ ]:
df_puntos = df_train.drop(columns=['distancia', 'vel alambre', 'voltaje', 'peri voltaje', 'flujo gas','corriente','ubicacion'])

In [ ]:
df_puntos

In [ ]:
# Merge de los puntos con los datos de entrenamiento
df = pd.merge(df, df_puntos, on='imagen', how='left')

In [ ]:
df.to_csv('datos/df.csv', index=False)

In [23]:
# Concatenar datasets
df_medio = pd.read_csv('datos/eliminar/matriz_grilla.csv')
df_inicio = pd.read_csv('datos/eliminar/matriz_grilla_inicio.csv')
df_final = pd.read_csv('datos/eliminar/matriz_grilla_final.csv')

In [24]:
df_salida = pd.concat([df_inicio, df_medio, df_final])

In [25]:
# Guardar dataset final
df_salida.to_csv('datos/df_salida.csv', index=False)

In [4]:
df = pd.read_csv('datos/df.csv')
df_salida = pd.read_csv('datos/df_salida.csv')

In [5]:
# Mergear ambos dataset df y df_salida
df_ = pd.merge(df, df_salida, on='imagen', how='left')


In [7]:
df_.to_csv('datos/df_.csv', index=False)

In [3]:
df = pd.read_csv('datos/matriz_grilla_final.csv')

In [11]:
df.matriz.shape

(18,)

In [14]:
salida = df.to_numpy().tolist()

In [18]:
len(salida[0][1])

130

In [19]:
salida[0][1]

'[[1, 0, 0, 0, 0, 0, 0, 1], [1, 0, 0, 0, 0, 0, 0, 1], [1, 0, 0, 0, 0, 0, 1, 0], [0, 1, 1, 1, 1, 1, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0]]'